# 6. Klassifikation - Modeling
#### Überwachtes Klassifikationsverfahren
## Anforderungen an Projektumsetzung: Klassifikation

---
**AUFGABE:**

- Führen Sie mit dem Algorithmus Ihrer Wahl eine Klassifikationsaufgabe auf Ihren Daten durch.
- Teilen Sie dazu zunächst die Daten auf, um Overfitting beim Trainieren des Algorithmus und bei der Parameterauswahl zu vermeiden. Erklären Sie die gewählte Strategie und die Größenverhältnisse.
- Wählen Sie geeignete Features aus und setzen Sie die Parameter des Algorithmus. Beschreiben Sie das gewälhte Vorgehen für die Auswahl der Features und Parameter. Berichten Sie den Parameterraum und die final gewählten Parameter. Geben Sie die Performanz auf den Trainingsdaten (bzw. Entwicklungsdaten, falls verwendet) an.
- Evaluieren Sie die Klassifikation auf den ungesehenen Testdaten. Betrachten Sie Precision und Recall sowie den F-Wert. Welches Maß ist für Ihre Anwendung wichtiger? Bewerten Sie Ihr Ergebnis. Ist es in der Praxis voraussichtlich zufriedenstellend?

In [1]:
# Imports für unten

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

In [2]:
apps = pd.read_csv("Daten/Google-Playstore_Edit2.csv")

In [3]:
# alle Kategorien löschen, die für uns als Unternehmen irrelevant und für eine Entwicklung ausgeschlossen sind
less_apps = apps[apps['Category'] == 'Action'] + apps[apps['Category'] == 'Arcade'] + apps[apps['Category'] == 'Beauty'] + apps[apps['Category'] == 'Casino'] + apps[apps['Category'] == 'Comics'] + apps[apps['Category'] == 'Dating'] + apps[apps['Category'] == 'Educational'] + apps[apps['Category'] == 'Puzzle'] + apps[apps['Category'] == 'Racing'] + apps[apps['Category'] == 'Role Playing'] + apps[apps['Category'] == 'Shopping'] + apps[apps['Category'] == 'Trivia'] + apps[apps['Category'] == 'Video Players & Editors'] 
            
apps.drop(less_apps.index, axis=0, inplace=True)            

In [4]:
cat = apps.groupby('Category')
cat['Category'].count()

Category
Adventure             23196
Art & Design          18538
Auto & Vehicles       18278
Board                 10588
Books & Reference    116726
Business             143761
Card                   8179
Casual                50797
Communication         48159
Education            241075
Entertainment        138268
Events                12839
Finance               65456
Food & Drink          73920
Health & Fitness      83501
House & Home          14369
Libraries & Demo       5196
Lifestyle            118324
Maps & Navigation     26722
Medical               32063
Music                  4202
Music & Audio        154898
News & Magazines      42804
Parenting              3810
Personalization       89210
Photography           35552
Productivity          79686
Simulation            23276
Social                44729
Sports                47478
Strategy               8525
Tools                143976
Travel & Local        67282
Weather                7245
Word                   8630
Name: Categ

In [5]:
# Datensatz random auf die Hälfte reduzieren und neues DataFrame erstellen
half_apps = apps.sample(frac = 0.5)

In [6]:
# Alle Spalten mit Unique-Werten werden gedropped - zu viel Rechenkapa notwendig
# apps_v2 = apps.copy()
half_apps.drop(columns=['App Name', 'App Id', 'Developer Id', 'Developer Website','Minimum Android', 'Developer Email', 'Privacy Policy', 'Released', 'Scraped Time', 'Last Updated'], inplace=True)

In [7]:
# Umwandlung in Float-Werte
half_apps['Free']             = half_apps['Free'].astype(float)
half_apps['Ad Supported']     = half_apps['Ad Supported'].astype(float)
half_apps['Editors Choice']   = half_apps['Editors Choice'].astype(float)
half_apps['In App Purchases'] = half_apps['In App Purchases'].astype(float)
half_apps['Maximum Installs'] = half_apps['Maximum Installs'].astype(float)

In [8]:
half_apps.dtypes

Category             object
Rating              float64
Rating Count        float64
Installs             object
Minimum Installs    float64
Maximum Installs    float64
Free                float64
Price               float64
Currency             object
Size                float64
Content Rating       object
Ad Supported        float64
In App Purchases    float64
Editors Choice      float64
Released Year       float64
dtype: object

In [9]:
# Selektion von den Spalten vom Typ object
half_apps = half_apps.dropna()
half_apps.select_dtypes(include=['object'])

,Category,Installs,Currency,Content Rating
2091952,Business,100+,USD,Teen
1078145,Finance,"5,000+",USD,Everyone
2273313,Tools,10+,USD,Everyone
1038383,Simulation,"100,000+",USD,Everyone
1434153,Education,10+,USD,Everyone
...,...,...,...,...
348272,Communication,"1,000+",USD,Everyone
590096,Education,"10,000+",USD,Everyone
1576720,Education,5+,USD,Everyone
747676,Productivity,"1,000+",USD,Everyone


In [10]:
# Aufteilung in Listen mit numerischen und mit noch kategorischen Werten
numerical_cols = list(half_apps.select_dtypes(include="float").columns)
categorical_cols = list(half_apps.select_dtypes(include="object").columns)

In [11]:
# Löschen von Category, da dies dann als Zielklasse verwendet werden soll
categorical_cols.remove("Category")
categorical_cols

['Installs', 'Currency', 'Content Rating']

In [12]:
# Da Klassifikation nur mit numerischen Daten funktioniert, werden mittels
# One-Hot-Endcoding aus den kategorischen Spalten, numerische Daten generiert
X_dumm = pd.get_dummies(half_apps[categorical_cols])

In [13]:
X_dumm.head(2)

,Installs_0+,Installs_1+,"Installs_1,000+","Installs_1,000,000+","Installs_1,000,000,000+",Installs_10+,"Installs_10,000+","Installs_10,000,000+",Installs_100+,"Installs_100,000+",...,Currency_USD,Currency_VND,Currency_XXX,Currency_ZAR,Content Rating_Adults only 18+,Content Rating_Everyone,Content Rating_Everyone 10+,Content Rating_Mature 17+,Content Rating_Teen,Content Rating_Unrated
2091952,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1078145,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0


In [14]:
# Zusammenfügen beider numerischen Listen
X = pd.concat([half_apps[numerical_cols], X_dumm], axis = 1)

In [15]:
X.head()

,Rating,Rating Count,Minimum Installs,Maximum Installs,Free,Price,Size,Ad Supported,In App Purchases,Editors Choice,...,Currency_USD,Currency_VND,Currency_XXX,Currency_ZAR,Content Rating_Adults only 18+,Content Rating_Everyone,Content Rating_Everyone 10+,Content Rating_Mature 17+,Content Rating_Teen,Content Rating_Unrated
2091952,0.0,0.0,100.0,173.0,1.0,0.0,28.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,1,0
1078145,3.3,36.0,5000.0,7805.0,1.0,0.0,4.0,1.0,1.0,0.0,...,1,0,0,0,0,1,0,0,0,0
2273313,0.0,0.0,10.0,47.0,1.0,0.0,6.4,1.0,0.0,0.0,...,1,0,0,0,0,1,0,0,0,0
1038383,3.4,1974.0,100000.0,146164.0,1.0,0.0,28.0,1.0,0.0,0.0,...,1,0,0,0,0,1,0,0,0,0
1434153,0.0,0.0,10.0,11.0,1.0,0.0,3.5,1.0,0.0,0.0,...,1,0,0,0,0,1,0,0,0,0


In [16]:
y = half_apps['Category'] #.copy()
print(f"X und y haben gleiche Anzahl: {X.shape[0] == y.shape[0]}")

X und y haben gleiche Anzahl: True


In [17]:
label_encoder = LabelEncoder()

In [18]:
y = label_encoder.fit_transform(y) # macht alles zu 0, 1, 2, 3 ...

In [19]:
# Daten in Trainings- und Test aufteilen
X_train, X_test1, y_train, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Testdaten in Entwicklung und echten Test aufteilen (50-50, stratifiziert)
X_dev, X_test, y_dev, y_test = train_test_split(X_test1, y_test1, test_size=0.5, stratify=y_test1, random_state=42)

- alles manuell statt gridsearch durchführen
- also 2x feature selection jeweilis mit None und mean, dann innerhalb dieser zwei feature selection jeweils test und traindaten auswerten --> In Summe 4 Ergebnisse
- also 2x classifier jeweilis mit linear und rbf, dann innerhalb dieser zwei classifier jeweils test und traindaten auswerten --> In Summe 4 Ergebnisse

### Vorhersagen auf den Trainingsdaten

In [21]:
# Parameter des Algorithmus setzen
# Mit SelectFromModel-Parameter threshold=None und SVC-Parameter kernel=rbf --> alles default-Werte

pipeline1 = Pipeline(steps=[
                ('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'))),
                ('classifier', SVC(kernel='rbf'))])

pipeline1.fit(X_train[:5000], y_train[:5000])

train_labels1 = pipeline1.predict(X_train[:5000])
      
print(classification_report(y_train[:5000], train_labels1[:5000]))

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        63
           1       0.00      0.00      0.00        35
           2       0.00      0.00      0.00        38
           3       0.00      0.00      0.00        28
           4       0.00      0.00      0.00       322
           5       0.00      0.00      0.00       387
           6       0.00      0.00      0.00        19
           7       0.25      0.02      0.04       128
           8       0.00      0.00      0.00       102
           9       0.12      1.00      0.22       615
          10       0.30      0.01      0.02       338
          11       0.00      0.00      0.00        33
          12       0.00      0.00      0.00       154
          13       0.00      0.00      0.00       182
          14       0.00      0.00      0.00       210
          15       0.00      0.00      0.00        31
          16       0.00      0.00      0.00        12
          17       1.00    

/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
# Parameter des Algorithmus setzen
# Mit SelectFromModel-Parameter threshold='mean' und SVC-Parameter kernel=rbf

pipeline2 = Pipeline(steps=[
                ('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'), threshold='mean')),
                ('classifier', SVC())])

pipeline2.fit(X_train[:5000], y_train[:5000])

train_labels2 = pipeline2.predict(X_train[:5000])
      
print(classification_report(y_train[:5000], train_labels2[:5000]))

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        63
           1       0.00      0.00      0.00        35
           2       0.00      0.00      0.00        38
           3       0.00      0.00      0.00        28
           4       0.14      0.08      0.10       322
           5       0.15      0.82      0.25       387
           6       0.00      0.00      0.00        19
           7       0.33      0.03      0.06       128
           8       0.00      0.00      0.00       102
           9       0.14      0.45      0.21       615
          10       0.42      0.01      0.03       338
          11       0.00      0.00      0.00        33
          12       0.00      0.00      0.00       154
          13       0.00      0.00      0.00       182
          14       0.00      0.00      0.00       210
          15       0.00      0.00      0.00        31
          16       0.00      0.00      0.00        12
          17       0.00    

/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
# Parameter des Algorithmus setzen
# Mit SelectFromModel-Parameter threshold='mean' und SVC-Parameter kernel='linear'

pipeline3 = Pipeline(steps=[
                ('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'), threshold='mean')),
                ('classifier', SVC(kernel='linear'))])

pipeline3.fit(X_train[:5000], y_train[:5000])

train_labels3 = pipeline3.predict(X_train[:5000])
      
print(classification_report(y_train[:5000], train_labels3[:5000]))

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.25      0.02      0.03        63
           1       0.00      0.00      0.00        35
           2       0.00      0.00      0.00        38
           3       0.00      0.00      0.00        28
           4       0.12      0.06      0.08       322
           5       0.15      0.81      0.25       387
           6       0.00      0.00      0.00        19
           7       0.29      0.02      0.03       128
           8       0.00      0.00      0.00       102
           9       0.13      0.44      0.21       615
          10       0.33      0.02      0.03       338
          11       0.00      0.00      0.00        33
          12       0.50      0.01      0.03       154
          13       0.00      0.00      0.00       182
          14       0.00      0.00      0.00       210
          15       0.00      0.00      0.00        31
          16       0.00      0.00      0.00        12
          17       0.09    

/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Parameter des Algorithmus setzen
# Mit SelectFromModel-Parameter threshold=None und SVC-Parameter kernel='linear' 
# Abbruch (nach 30 min) dieses Szenarios, aufgrund von zu langer Auswertung 
pipeline4 = Pipeline(steps=[
                ('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'), threshold=None)),
                ('classifier', SVC(kernel='linear'))])

pipeline4.fit(X_train[:5000], y_train[:5000])

train_labels4 = pipeline4.predict(X_train[:5000])
      de
print(classification_report(y_train[:5000], train_labels4[:5000]))

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Vorhersage auf den Entwicklungsdaten --> X_dev

In [24]:
# Parameter des Algorithmus setzen
# Mit SelectFromModel-Parameter threshold='mean' und SVC-Parameter kernel=rbf 

pipeline5 = Pipeline(steps=[
                ('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'), threshold='mean')),
                ('classifier', SVC())])

pipeline5.fit(X_train[:5000], y_train[:5000])

print("Default-Score des Klassifizierers: Accuracy=",pipeline5.score(X_test1, y_test1), "\n")

train_labels5 = pipeline5.predict(X_dev)
      
print(classification_report(y_dev, train_labels5))

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Default-Score des Klassifizierers: Accuracy= 0.14556192995678052 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1131
           1       0.00      0.00      0.00       907
           2       0.00      0.00      0.00       854
           3       0.00      0.00      0.00       506
           4       0.08      0.05      0.06      5768
           5       0.14      0.82      0.23      6843
           6       0.00      0.00      0.00       401
           7       0.16      0.01      0.02      2437
           8       0.00      0.00      0.00      2309
           9       0.13      0.43      0.20     11821
          10       0.14      0.00      0.01      6792
          11       0.00      0.00      0.00       616
          12       0.00      0.00      0.00      3112
          13       0.00      0.00      0.00      3564
          14       0.00      0.00      0.00      4049
          15       0.00      0.00      0.00       679
          16  

/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# Parameter des Algorithmus setzen
# Mit SelectFromModel-Parameter threshold=None und SVC-Parameter kernel=rbf --> alles default-Werte

pipeline6 = Pipeline(steps=[
                ('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'))),
                ('classifier', SVC(kernel='rbf'))])

pipeline6.fit(X_train[:5000], y_train[:5000])

print("Default-Score des Klassifizierers: Accuracy=",pipeline6.score(X_test1, y_test1), "\n")

train_labels6 = pipeline6.predict(X_dev)
      
print(classification_report(y_dev, train_labels6))

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Default-Score des Klassifizierers: Accuracy= 0.1214349067679735 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1131
           1       0.00      0.00      0.00       907
           2       0.00      0.00      0.00       854
           3       0.00      0.00      0.00       506
           4       0.00      0.00      0.00      5768
           5       0.00      0.00      0.00      6843
           6       0.00      0.00      0.00       401
           7       0.06      0.00      0.00      2437
           8       0.00      0.00      0.00      2309
           9       0.12      1.00      0.22     11821
          10       0.06      0.00      0.01      6792
          11       0.00      0.00      0.00       616
          12       0.00      0.00      0.00      3112
          13       0.00      0.00      0.00      3564
          14       0.00      0.00      0.00      4049
          15       0.00      0.00      0.00       679
          16   

/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
# Parameter des Algorithmus setzen
# Mit SelectFromModel-Parameter threshold='mean' und SVC-Parameter kernel='linear' 

pipeline7 = Pipeline(steps=[
                ('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'), threshold='mean')),
                ('classifier', SVC(kernel='linear'))])

pipeline7.fit(X_train[:5000], y_train[:5000])

print("Default-Score des Klassifizierers: Accuracy=",pipeline7.score(X_test1, y_test1), "\n")

train_labels7 = pipeline7.predict(X_dev)
      
print(classification_report(y_dev, train_labels7))

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Default-Score des Klassifizierers: Accuracy= 0.1455773105495486 

              precision    recall  f1-score   support

           0       0.14      0.01      0.02      1131
           1       0.00      0.00      0.00       907
           2       0.00      0.00      0.00       854
           3       0.00      0.00      0.00       506
           4       0.08      0.04      0.05      5768
           5       0.14      0.82      0.23      6843
           6       0.00      0.00      0.00       401
           7       0.16      0.00      0.01      2437
           8       0.02      0.00      0.00      2309
           9       0.13      0.43      0.20     11821
          10       0.16      0.01      0.01      6792
          11       0.00      0.00      0.00       616
          12       0.19      0.00      0.00      3112
          13       0.00      0.00      0.00      3564
          14       0.00      0.00      0.00      4049
          15       0.00      0.00      0.00       679
          16   

/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Parameter des Algorithmus setzen
# Mit SelectFromModel-Parameter threshold=None und SVC-Parameter kernel='linear'
# Abbruch (nach 30 min) dieses Szenarios, aufgrund von zu langer Auswertung 
pipeline8 = Pipeline(steps=[
                ('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'), threshold=None)),
                ('classifier', SVC(kernel='linear'))])

pipeline8.fit(X_train[:5000], y_train[:5000])

print("Default-Score des Klassifizierers: Accuracy=",pipeline8.score(X_test1, y_test1), "\n")

train_labels8 = pipeline8.predict(X_dev)
      
print(classification_report(y_dev, train_labels8))

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Auswertung Trainings- vs. Testdaten
- grundsätzlich haben die Trainingsdaten eine höhere Accuracy als die Entwicklungs- bzw. Testdaten
- bei den Vorhersagen auf den Trainings- und Testdaten schneiden die Werte jeweils dem SelectFromModel-Parameter threshold='mean' besser als also bei threshold=None
- mit den Parametern Threshold='mean' und kernel='linear' jedoch noch besser als mit kernel='rbf' (bei den Vorhersagen auf den Testdaten)
----
Trotz zufälliger Halbierung des Datensatzes, Eliminierung von NaN-Werten und Eingrenzung der Zielklassen, können aufgrund der zahlreichen Datensätze keine Schlüsse gezogen werden. Die Accuracy liegt bei allen Vorhersagen recht niedrig, da bei vielen Zielklassen die Precision und F-score `0.0` bewertet wurden, da zu den jeweiligen Zielklassen keine aussagekräftigen Trainings- oder Testdaten zugeordnet werden konnten.